In [1]:
import keras
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=12)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize

In [2]:
def make_train_test(data, test_size=0.3):
    sbd = data[data['sbj'] == 0]
    X_train, X_test, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
    for i in range(1, data.sbj.max()+1):
        sbd = data[data['sbj'] == i]
        tr, ts, y1, y2 = train_test_split(sbd, sbd, test_size=test_size, random_state=1111)
        X_train = pd.concat([X_train, tr])
        X_test = pd.concat([X_test, ts])
        
    return X_train.reset_index(drop=True), X_test.reset_index(drop=True)

In [3]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[1]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [4]:
data_df = pd.read_csv('../../_00_data/Tavares2017_prep.csv')

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [5]:
tr_df, ts_df = make_train_test(data_df, test_size=0.4)

obs_train = np.asarray(tr_df[['rt', 'choice', 'sbj']])
X_train = np.asarray(tr_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1']])

obs_test = np.asarray(ts_df[['rt', 'choice', 'sbj']])
X_test = np.asarray(ts_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1']])

print(X_train.shape, X_test.shape)

(18269, 4) (12195, 4)


In [6]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 1e-14)), axis=-1)

In [7]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [8]:
layers_neuron = [10, 10, 8, 5]
input_shape = 4
output_shape = 2
activation = 'tanh'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 5)                 45        
                                                                 
 out_layer (Out_layer)       (None, 2)                 87        
                                                                 
Total params: 380 (1.48 KB)
Trainable params: 380 (1.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [10]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=4000)

Epoch 1/4000
1/1 [==============================] - 1s 577ms/step - loss: 97629.1641
Epoch 2/4000
1/1 [==============================] - 0s 5ms/step - loss: 97124.7031
Epoch 3/4000
1/1 [==============================] - 0s 4ms/step - loss: 96760.7031
Epoch 4/4000
1/1 [==============================] - 0s 5ms/step - loss: 96457.3281
Epoch 5/4000
1/1 [==============================] - 0s 4ms/step - loss: 96189.3438
Epoch 6/4000
1/1 [==============================] - 0s 5ms/step - loss: 95945.0000
Epoch 7/4000
1/1 [==============================] - 0s 5ms/step - loss: 95717.7344
Epoch 8/4000
1/1 [==============================] - 0s 4ms/step - loss: 95503.5234
Epoch 9/4000
1/1 [==============================] - 0s 5ms/step - loss: 95299.6406
Epoch 10/4000
1/1 [==============================] - 0s 5ms/step - loss: 95104.1875
Epoch 11/4000
1/1 [==============================] - 0s 4ms/step - loss: 94915.8125
Epoch 12/4000
1/1 [==============================] - 0s 4ms/step - loss: 94733.5781

1/1 [==============================] - 0s 6ms/step - loss: 85330.2266
Epoch 99/4000
1/1 [==============================] - 0s 6ms/step - loss: 85248.5156
Epoch 100/4000
1/1 [==============================] - 0s 6ms/step - loss: 85167.2188
Epoch 101/4000
1/1 [==============================] - 0s 6ms/step - loss: 85086.3750
Epoch 102/4000
1/1 [==============================] - 0s 6ms/step - loss: 85005.9688
Epoch 103/4000
1/1 [==============================] - 0s 6ms/step - loss: 84926.0000
Epoch 104/4000
1/1 [==============================] - 0s 6ms/step - loss: 84846.4531
Epoch 105/4000
1/1 [==============================] - 0s 6ms/step - loss: 84767.3281
Epoch 106/4000
1/1 [==============================] - 0s 6ms/step - loss: 84688.6406
Epoch 107/4000
1/1 [==============================] - 0s 6ms/step - loss: 84610.3672
Epoch 108/4000
1/1 [==============================] - 0s 6ms/step - loss: 84532.5156
Epoch 109/4000
1/1 [==============================] - 0s 6ms/step - loss: 84455.0

1/1 [==============================] - 0s 6ms/step - loss: 79020.4844
Epoch 195/4000
1/1 [==============================] - 0s 6ms/step - loss: 78963.6328
Epoch 196/4000
1/1 [==============================] - 0s 6ms/step - loss: 78906.5547
Epoch 197/4000
1/1 [==============================] - 0s 6ms/step - loss: 78849.2344
Epoch 198/4000
1/1 [==============================] - 0s 6ms/step - loss: 78791.6875
Epoch 199/4000
1/1 [==============================] - 0s 6ms/step - loss: 78733.8750
Epoch 200/4000
1/1 [==============================] - 0s 5ms/step - loss: 78675.8125
Epoch 201/4000
1/1 [==============================] - 0s 5ms/step - loss: 78617.4688
Epoch 202/4000
1/1 [==============================] - 0s 6ms/step - loss: 78558.8516
Epoch 203/4000
1/1 [==============================] - 0s 6ms/step - loss: 78499.9844
Epoch 204/4000
1/1 [==============================] - 0s 5ms/step - loss: 78440.8359
Epoch 205/4000
1/1 [==============================] - 0s 6ms/step - loss: 78381.

1/1 [==============================] - 0s 6ms/step - loss: 72972.8750
Epoch 291/4000
1/1 [==============================] - 0s 6ms/step - loss: 72909.8984
Epoch 292/4000
1/1 [==============================] - 0s 6ms/step - loss: 72846.9844
Epoch 293/4000
1/1 [==============================] - 0s 6ms/step - loss: 72784.1719
Epoch 294/4000
1/1 [==============================] - 0s 6ms/step - loss: 72721.4297
Epoch 295/4000
1/1 [==============================] - 0s 6ms/step - loss: 72658.7969
Epoch 296/4000
1/1 [==============================] - 0s 6ms/step - loss: 72596.2422
Epoch 297/4000
1/1 [==============================] - 0s 5ms/step - loss: 72533.8125
Epoch 298/4000
1/1 [==============================] - 0s 6ms/step - loss: 72471.4453
Epoch 299/4000
1/1 [==============================] - 0s 5ms/step - loss: 72409.2109
Epoch 300/4000
1/1 [==============================] - 0s 6ms/step - loss: 72347.0391
Epoch 301/4000
1/1 [==============================] - 0s 6ms/step - loss: 72285.

1/1 [==============================] - 0s 6ms/step - loss: 67487.9844
Epoch 387/4000
1/1 [==============================] - 0s 6ms/step - loss: 67437.3359
Epoch 388/4000
1/1 [==============================] - 0s 6ms/step - loss: 67386.7656
Epoch 389/4000
1/1 [==============================] - 0s 6ms/step - loss: 67336.3594
Epoch 390/4000
1/1 [==============================] - 0s 6ms/step - loss: 67286.0000
Epoch 391/4000
1/1 [==============================] - 0s 6ms/step - loss: 67235.8125
Epoch 392/4000
1/1 [==============================] - 0s 6ms/step - loss: 67185.6953
Epoch 393/4000
1/1 [==============================] - 0s 6ms/step - loss: 67135.7422
Epoch 394/4000
1/1 [==============================] - 0s 5ms/step - loss: 67085.8438
Epoch 395/4000
1/1 [==============================] - 0s 6ms/step - loss: 67036.1094
Epoch 396/4000
1/1 [==============================] - 0s 6ms/step - loss: 66986.4375
Epoch 397/4000
1/1 [==============================] - 0s 6ms/step - loss: 66936.

...

1/1 [==============================] - 0s 6ms/step - loss: 43836.2422
Epoch 3528/4000
1/1 [==============================] - 0s 6ms/step - loss: 43841.4102
Epoch 3529/4000
1/1 [==============================] - 0s 6ms/step - loss: 43845.0938
Epoch 3530/4000
1/1 [==============================] - 0s 6ms/step - loss: 43857.7734
Epoch 3531/4000
1/1 [==============================] - 0s 6ms/step - loss: 43928.0781
Epoch 3532/4000
1/1 [==============================] - 0s 6ms/step - loss: 43895.6992
Epoch 3533/4000
1/1 [==============================] - 0s 6ms/step - loss: 43885.6250
Epoch 3534/4000
1/1 [==============================] - 0s 6ms/step - loss: 43865.1641
Epoch 3535/4000
1/1 [==============================] - 0s 12ms/step - loss: 43847.2148
Epoch 3536/4000
1/1 [==============================] - 0s 6ms/step - loss: 43837.3711
Epoch 3537/4000
1/1 [==============================] - 0s 6ms/step - loss: 43838.9531
Epoch 3538/4000
1/1 [==============================] - 0s 6ms/step - 

1/1 [==============================] - 0s 6ms/step - loss: 43841.3164
Epoch 3623/4000
1/1 [==============================] - 0s 6ms/step - loss: 43894.8711
Epoch 3624/4000
1/1 [==============================] - 0s 6ms/step - loss: 43848.2812
Epoch 3625/4000
1/1 [==============================] - 0s 6ms/step - loss: 43849.4805
Epoch 3626/4000
1/1 [==============================] - 0s 6ms/step - loss: 43851.3398
Epoch 3627/4000
1/1 [==============================] - 0s 6ms/step - loss: 43846.1562
Epoch 3628/4000
1/1 [==============================] - 0s 6ms/step - loss: 43842.9922
Epoch 3629/4000
1/1 [==============================] - 0s 6ms/step - loss: 43848.4531
Epoch 3630/4000
1/1 [==============================] - 0s 6ms/step - loss: 43848.8984
Epoch 3631/4000
1/1 [==============================] - 0s 6ms/step - loss: 43902.7930
Epoch 3632/4000
1/1 [==============================] - 0s 6ms/step - loss: 43848.6328
Epoch 3633/4000
1/1 [==============================] - 0s 6ms/step - l

1/1 [==============================] - 0s 7ms/step - loss: 43832.1875
Epoch 3718/4000
1/1 [==============================] - 0s 7ms/step - loss: 43825.4609
Epoch 3719/4000
1/1 [==============================] - 0s 6ms/step - loss: 43879.0859
Epoch 3720/4000
1/1 [==============================] - 0s 7ms/step - loss: 43833.2969
Epoch 3721/4000
1/1 [==============================] - 0s 6ms/step - loss: 43838.5859
Epoch 3722/4000
1/1 [==============================] - 0s 7ms/step - loss: 43869.5234
Epoch 3723/4000
1/1 [==============================] - 0s 6ms/step - loss: 43911.7656
Epoch 3724/4000
1/1 [==============================] - 0s 7ms/step - loss: 43850.6172
Epoch 3725/4000
1/1 [==============================] - 0s 6ms/step - loss: 43841.5195
Epoch 3726/4000
1/1 [==============================] - 0s 7ms/step - loss: 43833.7109
Epoch 3727/4000
1/1 [==============================] - 0s 7ms/step - loss: 43832.0391
Epoch 3728/4000
1/1 [==============================] - 0s 6ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 43831.3672
Epoch 3813/4000
1/1 [==============================] - 0s 7ms/step - loss: 43882.0508
Epoch 3814/4000
1/1 [==============================] - 0s 6ms/step - loss: 43841.5000
Epoch 3815/4000
1/1 [==============================] - 0s 6ms/step - loss: 43839.4141
Epoch 3816/4000
1/1 [==============================] - 0s 7ms/step - loss: 43844.7656
Epoch 3817/4000
1/1 [==============================] - 0s 6ms/step - loss: 43846.9062
Epoch 3818/4000
1/1 [==============================] - 0s 6ms/step - loss: 43847.5625
Epoch 3819/4000
1/1 [==============================] - 0s 6ms/step - loss: 43849.4609
Epoch 3820/4000
1/1 [==============================] - 0s 7ms/step - loss: 43844.6250
Epoch 3821/4000
1/1 [==============================] - 0s 7ms/step - loss: 43840.1250
Epoch 3822/4000
1/1 [==============================] - 0s 8ms/step - loss: 43829.8984
Epoch 3823/4000
1/1 [==============================] - 0s 6ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 43838.6523
Epoch 3908/4000
1/1 [==============================] - 0s 6ms/step - loss: 43831.8438
Epoch 3909/4000
1/1 [==============================] - 0s 6ms/step - loss: 43825.3555
Epoch 3910/4000
1/1 [==============================] - 0s 6ms/step - loss: 43822.5742
Epoch 3911/4000
1/1 [==============================] - 0s 6ms/step - loss: 43862.9258
Epoch 3912/4000
1/1 [==============================] - 0s 6ms/step - loss: 43819.8555
Epoch 3913/4000
1/1 [==============================] - 0s 6ms/step - loss: 43814.5000
Epoch 3914/4000
1/1 [==============================] - 0s 6ms/step - loss: 43822.0781
Epoch 3915/4000
1/1 [==============================] - 0s 6ms/step - loss: 43821.1797
Epoch 3916/4000
1/1 [==============================] - 0s 6ms/step - loss: 43828.4297
Epoch 3917/4000
1/1 [==============================] - 0s 6ms/step - loss: 43833.4570
Epoch 3918/4000
1/1 [==============================] - 0s 6ms/step - l

In [11]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

382/382 [==============================] - 0s 318us/step


In [12]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df.item_value_1[s] - ts_df.item_value_0[s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)

In [13]:
pred_bhv.to_csv('Tv2017_out_of_sample.csv')
ts_df.to_csv('Tv2017_test.csv')